In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = [9, 5]

# Problem 1: Tensor contractions

In [ ]:
dim = 20
A = np.random.random([dim]*4)
B = np.random.random([dim]*4)
A.shape, B.shape

In [ ]:
# to get the tensors in a matrix form, we can use a reshape:
print("Matrix shape:", A.reshape(dim, dim**3).shape)
# of course the content doesn't change:
assert np.all(A.reshape(dim, dim**3).flatten() == A.flatten())

# to get the legs in the right order, we can use transpose
test = np.random.random([1, 2, 3, 4, 5])
print("Initial shape:", test.shape)
print("Reordered shape:", test.transpose(3, 2, 0, 1, 4).shape)

Let's contract the tensors A and B using a fast BLAS matrix multiplication
(Check ``np.show_config()`` that you have BLAS for speed ups).

The contraction we consider equals the sum
$$ \sum_{mn} A_{imjn} B_{mkln}.$$

In [ ]:
Atrans = A.transpose(0, 2, 1, 3)
Amat = Atrans.reshape(dim**2, dim**2)
Btrans = B.transpose(0, 3, 1, 2)
Bmat = Btrans.reshape(dim**2, dim**2)

# contraction:
ABmat = Amat @ Bmat

# bring it back in tensor form
AB = ABmat.reshape([dim]*4)
AB.shape

In [ ]:
# Let's check our result using np.einsum
AB_einsum = np.einsum('imjn,mkln', A, B)
assert np.allclose(AB, AB_einsum)

In [ ]:
# While einsum is convenient, a matrix product is much faster
%timeit A.transpose(0, 2, 1, 3).reshape(dim**2, dim**2) @ B.transpose(0, 3, 1, 2).reshape(dim**2, dim**2)
%timeit np.einsum('imjn,mkln', A, B)
# on my machine 50 times for this example:
# 2.51 ms ± 35.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
# 108 ms ± 1.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

As we see, it is not hard to contract networks using fast linear algebra libraries, however it is cumbersome to handle the bookkeeping.

Therefore, one should use appropriate libraries.
A common notation is `ncon`:

In [ ]:
from ncon import ncon

AB_ncon = ncon([A, B], [(-1, 1, -2, 2), (1, -3, -4, 2)])
assert np.allclose(AB, AB_ncon)

In [ ]:
# We see, ncon also is performant
%timeit ncon([A, B], [(-1, 1, -2, 2), (1, -3, -4, 2)])
# 2.83 ms ± 351 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

# Problem 2: Compression

In [ ]:
from scipy import misc
face = misc.face(gray=True)
plt.imshow(face, cmap='gray');

In [ ]:
u, s, vh = np.linalg.svd(face, full_matrices=False)
assert np.allclose((u*s)@vh, face)
plt.plot(s/s[0])
plt.yscale('log')

In [ ]:
# truncate the singular values for compression
num = 150
__, axes = plt.subplots(ncols=2)
axes[0].set_title("original")
axes[0].imshow(face, cmap="gray")
axes[1].set_title("compressed")
axes[1].imshow((u*s)[:, :num] @ vh[:num, :], cmap="gray");

In [ ]:
# the error is given by the truncated singular values
# Frobenius norm
ferr_singular = np.sqrt(np.sum(s[num:]**2))
ferr_norm = np.linalg.norm(face - (u*s)[:, :num] @ vh[:num, :], ord='fro')
print("Frobenius norm:", ferr_singular, ferr_norm)
# Spectral norm
serr_singular = s[num]
serr_norm = np.linalg.norm(face - (u*s)[:, :num] @ vh[:num, :], ord=2)
print("Spectral norm:", serr_singular, serr_norm)

In [ ]:
# compression
((u*s)[:, :num].size + vh[:num, :].size) / face.size

Note, that physical quantities can oftentimes be compressed much better.
Some example quantity:

In [ ]:
drange = np.arange(12) + 1
Mat = np.sqrt(drange[:, None, None, None, None]
              + 2*drange[None, :, None, None, None]
              + 3*drange[None, None, :, None, None]
              + 4*drange[None, None, None, :, None]
              + 5*drange[None, None, None, None, :]
             )
s = np.linalg.svd(Mat.reshape(12**2, 12**3), compute_uv=False)
plt.plot(s/s[0])
plt.yscale('log')

Let's also compress the colored image

In [ ]:
face = misc.face(gray=False)
plt.imshow(face);

In [ ]:
# we can try to truncate each color
red = face[..., 0]
green = face[..., 1]
blue = face[..., 2]
rsvd = np.linalg.svd(red, full_matrices=False)
gsvd = np.linalg.svd(green, full_matrices=False)
bsvd = np.linalg.svd(blue, full_matrices=False)

plt.plot(rsvd[1], color='red')
plt.plot(gsvd[1], color='green')
plt.plot(bsvd[1], color='blue')
plt.yscale('log')

In [ ]:
num = 150
rtrunc = (rsvd[0]*rsvd[1])[:, :num]@rsvd[2][:num, :]
gtrunc = (gsvd[0]*gsvd[1])[:, :num]@gsvd[2][:num, :]
btrunc = (bsvd[0]*bsvd[1])[:, :num]@bsvd[2][:num, :]
trunc = np.stack([rtrunc, gtrunc, btrunc], axis=-1)

__, axes = plt.subplots(ncols=2)
axes[0].set_title("original")
axes[0].imshow(face)
axes[1].set_title("compressed")
axes[1].imshow(trunc.astype(int));
print("Compression:",
      3*((rsvd[0]*rsvd[1])[:, :num].size + rsvd[2][:num, :].size) / face.size)

In [ ]:
# or, we use a reshape
shape = face.shape
u, s, vh = np.linalg.svd(face.reshape(shape[0], shape[1]*shape[2]),
                         full_matrices=False)
plt.plot(s/s[0])
plt.yscale('log')

In [ ]:
num = 150
__, axes = plt.subplots(ncols=2)
axes[0].set_title("original")
axes[0].imshow(face)
axes[1].set_title("compressed")
axes[1].imshow(((u*s)[:, :num] @ vh[:num, :]).reshape(shape).astype(int))
print("Compression:", ((u*s)[:, :num].size + (vh)[:num, :].size )/face.size)